In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Milvus
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import os
import ipywidgets as widgets
from IPython.display import display
from dotenv import load_dotenv

In [2]:
load_dotenv()  # take environment variables from .env.
openai_api_key = os.getenv('OPENAI_API_KEY')
milvus_api_key = os.getenv('MILVUS_API_KEY')

connection_args={
        "uri": "https://in03-5052868020ac71b.api.gcp-us-west1.zillizcloud.com",
        "user": "vaclav@pechtor.ch",
        "token": milvus_api_key,
        "secure": True
    }


In [5]:
directory = './datasets/'
docs = []

In [6]:
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    loader = TextLoader(filepath)
    doc = loader.load()
    docs.extend(doc)

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
docs = text_splitter.split_documents(docs)

Created a chunk of size 1162, which is longer than the specified 1024
Created a chunk of size 1239, which is longer than the specified 1024
Created a chunk of size 1081, which is longer than the specified 1024
Created a chunk of size 1078, which is longer than the specified 1024
Created a chunk of size 1173, which is longer than the specified 1024
Created a chunk of size 3424, which is longer than the specified 1024
Created a chunk of size 3257, which is longer than the specified 1024
Created a chunk of size 1985, which is longer than the specified 1024
Created a chunk of size 2477, which is longer than the specified 1024
Created a chunk of size 1078, which is longer than the specified 1024
Created a chunk of size 1173, which is longer than the specified 1024


In [9]:
embeddings = OpenAIEmbeddings()

vector_store = Milvus.from_documents(
    docs,
    embedding=embeddings,
    connection_args=connection_args,
    collection_name='Simple_DIIN_Chat'
)

In [10]:
# Create text widget for input
input_text = widgets.Text()

# Create a global variable to hold the input
query = ""

# Create function to update the query variable
def update_query(change):
    global query
    query = change.new

# Tell the text widget to call update_query() function when its value changes
input_text.observe(update_query, 'value')

# Display the text widget
display(input_text)

Text(value='')

In [12]:
# query = "Was ist SAFe?"
print(f"Query: {query}")
docs = vector_store.similarity_search(query)

print(docs)

Query: Was ist das UCC Projekt?
[Document(page_content='Die Digitalisierung ist keinem kurzen oder längeren Trend unterworfen. Sie ist ein fortlaufender Prozess, der unseren Alltag beeinflusst. Abläufe werden einfacher, effizienter und schneller.\xa0\nDas Projekt «integrierte Kommunikations- und Kollaborationslösung», kurz UCC, wird die bestehende Telefonanlage in der kantonalen Verwaltung ablösen. Die dabei eingesetzte Software Skype fasst verschiedene Kommunikationsmöglichkeiten zusammen. Die bisherigen Möglichkeiten der Kommunikation bleiben bestehen. Skype eröffnet aber zusätzliche Varianten an Kommunikationsformen wie etwa die Chat-Funktion oder die Möglichkeit von Videokonferenzen.', metadata={'source': './datasets/informatik.lu.ch__.txt'}), Document(page_content='Die Digitalisierung ist keinem kurzen oder längeren Trend unterworfen. Sie ist ein fortlaufender Prozess, der unseren Alltag beeinflusst. Abläufe werden einfacher, effizienter und schneller.\xa0\nDas Projekt «integriert

In [21]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4',temperature=0.0)
qa = RetrievalQA.from_chain_type(llm=llm, 
                                 retriever=vector_store.as_retriever(),
                                 chain_type='stuff',
                                 return_source_documents=True)
result = qa({"query": query})

In [23]:
print(f'Query: {query}')
print(f'Result: {result}')

Query: Was ist das UCC Projekt?
Result: {'query': 'Was ist das UCC Projekt?', 'result': 'Das UCC-Projekt (Unified Communications & Collaboration) ist ein Projekt, das die Ablösung der bestehenden Telefonanlage in der kantonalen Verwaltung durch eine integrierte Kommunikations- und Kollaborationslösung zum Ziel hat. Die dabei eingesetzte Software Skype fasst verschiedene Kommunikationsmöglichkeiten zusammen und eröffnet zusätzliche Varianten an Kommunikationsformen wie Chat-Funktion, Videokonferenzen, Desktop Sharing und Collaboration-Funktionen. Dies soll die Online-Zusammenarbeit innerhalb der Verwaltung und mit externen Stellen vereinfachen und die Effizienz steigern. Vor der Einführung von Skype erhalten alle Mitarbeitenden eine ausführliche Schulung mittels E-Learning, und das Projekt wird durch die Fachstelle Change Management begleitet.', 'source_documents': [Document(page_content='Die Digitalisierung ist keinem kurzen oder längeren Trend unterworfen. Sie ist ein fortlaufender Pr